In [ ]:
# default_exp docstring

# Exporting Docstrings

> Converts `docment` docstrings to Numpy styled

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

The goal of this module is to take code that looks like the following:

In [ ]:
def addition(
    a:int, # The first number to add
    b:int=2, # The second number to add
) -> int: # The sum of a and b
    "Adds two numbers together"
    return a+b

And convert it to be the following:

In [ ]:
def addition(a,b) -> int:
    """Adds two numbers together
    
    Parameters
    ---------
    a : int
        The first number to add
    b : int
        The second number to add
        
    Returns
    -------
    int
        The sum of a and b
    """
    return a + b

In [ ]:
# TESTING CODE
import nbdev.export as exp
nb = exp.read_nb('99_test.ipynb')
default = exp.find_default_export(nb['cells'])
mod = exp.get_nbdev_module()
exports = [exp.is_export(c, default) for c in nb['cells']]
cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
flag_lines, code_lines = exp.split_flags_and_code(cells[0][1])
code_lines = exp._deal_import(code_lines, 'docstring.py')

In [ ]:
# TESTING CODE
code_lines

['def addition(',
 '    a:int, # The first number to add',
 '    b:int, # The second number to add',
 ') -> int: # The sum of a and b',
 '    "Adds two numbers together"',
 '    return a+b']

In [ ]:
#export
import inspect, ast, astunparse
from __future__ import annotations
import fastcore.docments as dments

Below we have an example string repsentation of the above docments style:

In [ ]:
source = """def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    return a+b
"""

In [ ]:
#export
def get_annotations(
    source:str # Source code of function or class
):
    "Extracts the type annotations from source code"
    parse = ast.parse(source)
    arg_annos = []
    for i,anno in enumerate(parse.body[0].args.args):
        if anno.annotation is not None:
            arg_annos.append(astunparse.unparse(anno.annotation).strip('\n'))
        else:
            arg_annos.append(anno.annotation)
        parse.body[0].args.args[i].annotation = None
    if parse.body[0].returns is not None:
        ret_anno = astunparse.unparse(parse.body[0].returns).strip('\n')
    else:
        ret_anno = None
    return arg_annos, ret_anno

In [ ]:
test_eq(get_annotations(source), (['(int, float)', 'int'], '(int, float)'))

In [ ]:
#export
def _get_leading(o):
    return len(o) - len(o.lstrip(o[0])), o[0]

In [ ]:
test_eq(_get_leading('  Hello my name is Zach'), (2, ' '))

In [ ]:
parsed_source.body[0].body[0].value

In [ ]:
#export
def reformat_function(
    source:str, # Source code
):
    "Takes messy source code and refactors it into a readable PEP-8 standard style"
    docs = dments.docments(source)
    annos = get_annotations(source)
    parsed_source = ast.parse(source)
    for i in range(len(parsed_source.body[0].args.args)):
        parsed_source.body[0].args.args[i].annotation = None
    parsed_source.body[0].returns = None
    unparsed_source = astunparse.unparse(parsed_source).lstrip('\n').split('\n')
    function_definition = unparsed_source[0]
    # Check if we have a docstring
    if isinstance(parsed_source.body[0].body[0].value, ast.Str):
        function_innards = "\n".join(unparsed_source[2:])
    else:
        function_innards = "\n".join(unparsed_source[1:])
    def _get_whitespace(): return whitespace_char*num_whitespace
    
    num_whitespace, whitespace_char = _get_leading(unparsed_source[2])
    docstring = f'\n{_get_whitespace()}"""\n'
    if isinstance(parsed_source.body[0].body[0].value, ast.Str):
        _quotes = ("'", '"')
        docstring += f'{_get_whitespace()}{unparsed_source[1].lstrip(whitespace_char).strip(_quotes[0]).strip(_quotes[1])}\n\n'
    docstring += f'{_get_whitespace()}Parameters\n'
    docstring += f'{_get_whitespace()}----------\n'
    for i, param in enumerate(docs.keys()):
        if param != "return" and param != "self":
            docstring += f'{_get_whitespace()}{param} : {annos[0][i]}\n'
            docstring += f'{whitespace_char * (num_whitespace+2)}{docs[param]}\n'
    if (annos[-1] != inspect._empty) and ('return' in docs.keys()):
        docstring += f'\n{_get_whitespace()}Returns\n'
        docstring += f'{_get_whitespace()}-------\n'
        docstring += f'{_get_whitespace()}{annos[1]}\n'
        docstring += f'{whitespace_char * (num_whitespace+2)}{docs["return"]}\n'
    docstring += f'{_get_whitespace()}"""\n'
    return f'{function_definition}{docstring}{function_innards}'

In [ ]:
print(source)

def __init__(
        self,
        a:int,
        b:(int, float), # The second number to use
    ):
        self.a = a
        self.b = b


In [ ]:
print(reformat_function(source))

def __init__(self, a, b):
    """
    Parameters
    ----------
    a : int
      None
    b : (int, float)
      The second number to use
    """
    self.a = a
    self.b = b



In [ ]:
source = """def __init__(
        self,
        a:int,
        b:(int, float), # The second number to use
    ):
        self.a = a
        self.b = b"""

In [ ]:
print(reformat_function(source))

def __init__(self, a, b):
    """
    Parameters
    ----------
    a : int
      None
    b : (int, float)
      The second number to use
    """
    self.a = a
    self.b = b



For classes we'll need to extract functions and constants, and only reformat the functions in each class

In [ ]:
class ExampleClass(object):
    """The summary line for a class docstring should fit on one line.

    If the class has public attributes, they may be documented here
    in an ``Attributes`` section and follow the same formatting as a
    function's ``Args`` section. Alternatively, attributes may be documented
    inline with the attribute's declaration (see __init__ method below).

    Properties created with the ``@property`` decorator should be documented
    in the property's getter method.

    Attributes
    ----------
    attr1 : str
        Description of `attr1`.
    attr2 : :obj:`int`, optional
        Description of `attr2`.

    """

    def __init__(self, param1, param2, param3):
        """Example of docstring on the __init__ method.

        The __init__ method may be documented in either the class level
        docstring, or as a docstring on the __init__ method itself.

        Either form is acceptable, but the two should not be mixed. Choose one
        convention to document the __init__ method and be consistent with it.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1 : str
            Description of `param1`.
        param2 : :obj:`list` of :obj:`str`
            Description of `param2`. Multiple
            lines are supported.
        param3 : :obj:`int`, optional
            Description of `param3`.

        """
        self.attr1 = param1
        self.attr2 = param2
        self.attr3 = param3  #: Doc comment *inline* with attribute

        #: list of str: Doc comment *before* attribute, with type specified
        self.attr4 = ["attr4"]

        self.attr5 = None
        """str: Docstring *after* attribute, with type specified."""

    @property
    def readonly_property(self):
        """str: Properties should be documented in their getter method."""
        return "readonly_property"

    @property
    def readwrite_property(self):
        """:obj:`list` of :obj:`str`: Properties with both a getter and setter
        should only be documented in their getter method.

        If the setter method contains notable behavior, it should be
        mentioned here.
        """
        return ["readwrite_property"]

    @readwrite_property.setter
    def readwrite_property(self, value):
        value

    def example_method(self, param1, param2):
        """Class methods are similar to regular functions.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1
            The first parameter.
        param2
            The second parameter.

        Returns
        -------
        bool
            True if successful, False otherwise.

        """
        return True

    def __special__(self):
        """By default special members with docstrings are not included.

        Special members are any methods or attributes that start with and
        end with a double underscore. Any special member with a docstring
        will be included in the output, if
        ``napoleon_include_special_with_doc`` is set to True.

        This behavior can be enabled by changing the following setting in
        Sphinx's conf.py::

            napoleon_include_special_with_doc = True

        """
        pass

    def __special_without_docstring__(self):
        pass

    def _private(self):
        """By default private members are not included.

        Private members are any methods or attributes that start with an
        underscore and are *not* special. By default they are not included
        in the output.

        This behavior can be changed such that private members *are* included
        by changing the following setting in Sphinx's conf.py::

            napoleon_include_private_with_doc = True

        """
        pass

    def _private_without_docstring(self):
        pass